# VRP Assignment - María Ármann

In [1]:
import numpy as np
import pandas as pd
import simpy
import random
import math
import matplotlib.pyplot as plt
import matplotlib
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import vrplib
import time
import random 
import copy

### Reading the data

We will be storing all eight instances in a list of instances. Throughout the code we will be iterating over the instances to get separete results for them all. 

In [2]:
import requests
import io
from pathlib import Path

import numpy as np
import pandas as pd
import vrplib
import requests

instances = []


Paths = ["ORTEC-VRPTW-ASYM-0bdff870-d1-n458-k35.txt",
         "ORTEC-VRPTW-ASYM-00c5356f-d1-n258-k12.txt", 
         "ORTEC-VRPTW-ASYM-0dc59ef2-d1-n213-k25.txt",
         "ORTEC-VRPTW-ASYM-04c694cd-d1-n254-k18.txt",
         "ORTEC-VRPTW-ASYM-08d8e660-d1-n460-k42.txt",
         "ORTEC-VRPTW-ASYM-0797afaf-d1-n313-k20.txt",
         "ORTEC-VRPTW-ASYM-02182cf8-d1-n327-k20.txt",
         "ORTEC-VRPTW-ASYM-01829532-d1-n324-k22.txt"]


for i in range(0,8):
    # Get instance from Github gist and write to file
    instance_loc = Path(Paths[i])
    instance = vrplib.read_instance(instance_loc)
    instances.append(instance)

The relevant items are:
- `capacity`: the vehicle capacity
- `edge_weight`: the distances matrix 
- `node_coord`: the location coordinates
- `demand`: the demand at each location
- `depot`: the depot index
- `service_time`: the service time at each location
- `deadline`: the customer deadlines
- `release_time`: the customer release times

## Heuristic Algorithm

For the heuristic algorithm we will be using the `Clark and Wright with Savings` algorithm. This is a simple and efficient algorithm and can easily be extended to include conditional constraints, like in our case: including the release dates and deadlines. 

### Step 1: Calculate the savings

The savings are calculated by using the following formula:
$$
s(i,j) = d(D,i) + d(j,D) - d(i,j)
$$
where $i \neq j$ and D is the depot.

In [3]:
#Function to calculate the savings for customers (i,j) where i != j

def Savings(instance):
    n = int(instance['dimension'])

    #Calculating the savings for all i,j
    savings = {}
    for i in range(1,n):
        for j in range(1,n):
            if i != j:
                s = (instance['edge_weight'][0][i]) + (instance['edge_weight'][j][0]) - (instance['edge_weight'][i][j])
                #s = (instance['edge_weight'][0][i]) + (instance['edge_weight'][0][j]) - (instance['edge_weight'][i][j])
                key = ("(" + str(i) + "," + str(j) + ")")
                savings[key] = s

    return savings



### Step 2: Sort the savings in decreasing order

In [4]:
def sortingSavings(savings):
    sortedSavings = dict(sorted(savings.items(), key=lambda x:x[1], reverse=True))
    
    return sortedSavings

### Step 3: Initialisation and constructing routes such that we do not break the constraints

In [130]:
def CWHeuristic(instance):
    
    n = int(instance['dimension'])
    
    savings = Savings(instance)
    sorted_savings = sortingSavings(savings)

    #Initialize all the routes such that all customers belong to separete routes
    #Dictionary where the key is the customer and the value is the route it belongs to
    routes = {i:[0, i, 0] for i in range(1, n)}
    newroutes = []
    assignedCust = []
    NotendNode = []

    # Iterate over the sorted savings dictionary
    starttime = time.time()
    for key, saving in sorted_savings.items() :
        i, j = map(int, key.strip('()').split(','))
        merging = True

        # Check if i and j are assigned to different routes and j is currently not in a route
        if (routes[i] != routes[j]) and (j not in assignedCust) and (i not in NotendNode):
        
            #We create the route and then do some checks if our route is feasible
            if routes[i] in newroutes:
                mergedRoute = routes[i][0:-1] + routes[j][1:]
            else:
                mergedRoute = routes[i][0:-1] + routes[j][1:]

            #Calculate the total demand of the merged route 
            totalDemand = 0

            for customer in mergedRoute[1:-1]:

                if merging == False:
                    break

                demand = instance['demand'][customer]
                totalDemand = totalDemand + demand

                #Check if by adding them to the same route we do not exceed capacity of the vehicles
                if (totalDemand) < (instance['capacity']+1):

                    #Calculate the latest release time of the new merged route:
                    latestRelease = 0
                    for customer in mergedRoute[1:-1]:
                        if instance['release_time'][customer] > latestRelease:
                            latestRelease = instance['release_time'][customer]

                    #Check if we can meet all the deadlines:
                    currTime = latestRelease #start time
                    from_Node = 0 #start driving from the depot

                    for customer in mergedRoute[1:-1]:
                        to_Node = customer

                        #If there is a customer where we can not meet the deadline then we don't merge and break 
                        if (currTime + instance['edge_weight'][from_Node][to_Node]) > instance['deadline'][customer]:
                            merging = False
                            break

                        #Can we arrive at the customer before the deadline?
                        else:
                            #If yes then we update the current Time. We add the driving distance and the service time
                            currTime = currTime + instance['edge_weight'][from_Node][to_Node] + instance['service_time'][to_Node]

                            #Update locations: from_Node
                            from_Node = customer

            if merging == True:
            #If we get through the loop then all deadlines are met and we merge the routes
                if routes[i] in newroutes:
                    newroutes.remove(routes[i])
                newroutes.append(mergedRoute)

                routes[i] = mergedRoute
                routes[j] = mergedRoute

                #i is not a end node anymore:
                NotendNode.append(i)

                #Customer has been assigned to a route
                assignedCust.append(j)
                assignedCust.append(i)

    allAssignedCustomers = []
    for route in newroutes:
        for customer in route[1:-1]:
            allAssignedCustomers.append(customer)


    for i in range(1,instance['dimension']):
        if i not in allAssignedCustomers:
            newroutes.append(routes[i])

    endtime = time.time()
    print("The code took:", endtime - starttime, "seconds to run.")

    return newroutes

### Running the Heuristic

In [131]:
#We will be storing our results from all instances in a list
routesInstances = []

for instance in instances: 
    newroutes = CWHeuristic(instance)
    routesInstances.append(newroutes)

The code took: 0.03769493103027344 seconds to run.
The code took: 0.03137612342834473 seconds to run.
The code took: 0.04349374771118164 seconds to run.
The code took: 0.049278974533081055 seconds to run.
The code took: 0.026233196258544922 seconds to run.
The code took: 0.029489755630493164 seconds to run.
The code took: 0.020919084548950195 seconds to run.
The code took: 0.01831817626953125 seconds to run.


The code prints the time it takes to run for each instance. It can be seen clearly that all instances run within a second. Due to this fact we decide that there is no need to implement a stopping critera of 60 seconds. 

In [13]:
#The following code prints how many vehicles are needed in each instance
for routes in routesInstances: 
    print("The total number of vehicles used to serve all customers:", len(routes))

The total number of vehicles used to serve all customers: 50
The total number of vehicles used to serve all customers: 43
The total number of vehicles used to serve all customers: 62
The total number of vehicles used to serve all customers: 64
The total number of vehicles used to serve all customers: 43
The total number of vehicles used to serve all customers: 40
The total number of vehicles used to serve all customers: 45
The total number of vehicles used to serve all customers: 41


### Validation Code - Heuristic 

The following code can be used to check if a certain route in a certain solution meets the time constraints. 

In [24]:
latestRelease = 0
for customer in newroutes[13][1:-1]:
#for customer in testroute[1:-1]:
    
    if instance['release_time'][customer] > latestRelease:
        latestRelease = instance['release_time'][customer]

    
currTime = latestRelease #start time
from_Node = 0 #start driving from the depot

for customer in newroutes[13][1:-1]:
#for customer in testroute[1:-1]:


    to_Node = customer
    
    #print("ReLTime", instance['release_time'][customer])
    
    print("For customer", customer, "we leave at time", currTime, "and arrive at ", currTime + instance['edge_weight'][from_Node][to_Node])
    print("The service time is: ", instance['service_time'][customer], "so we leave at",currTime + instance['edge_weight'][from_Node][to_Node]+instance['service_time'][customer])
    print("with the DLine being: ", instance['deadline'][customer])

    currTime = currTime + instance['edge_weight'][from_Node][to_Node] + instance['service_time'][to_Node]
                        
    #Update locations: from_Node
    from_Node = customer
    
    
print(latestRelease)

For customer 79 we leave at time 14400 and arrive at  18034
The service time is:  540 so we leave at 18574
with the DLine being:  19500
For customer 67 we leave at time 18574 and arrive at  26309
The service time is:  360 so we leave at 26669
with the DLine being:  22500
For customer 83 we leave at time 26669 and arrive at  30648
The service time is:  420 so we leave at 31068
with the DLine being:  33900
14400


## Metaheuristic Algorithm

For the metaheuristic algorithm we will be using the Tabu Search metaheuristic and we will use our previous obtained solution from the heuristic algorithm as the initialized feasible solution. This initialized solution respects all the constraints of the problem and is therefore a suitable fit as a starting point for the Tabu Search.  

### Step 1: Define the Neighbourhoods

The first step is to define the neighborhoods that will be used to generate new solutions in the Tabu Search algorithm. We will define the following neighborhoods: swap two customers between different routes, move a customer from one route to another, switch customers within a route and moving an end customer between routes. 

In [136]:
def create_neighbourhood(solution):
    #print("We get in", solution[0:5])
    
    original_sol = copy.deepcopy(solution)
    testingsol = copy.deepcopy(solution)
    
    #We will choose randomly what operation we will do 
    choice = random.choice([1,2,3,4])
        
    #If 1 then swap two random customers from two random routes 
    if choice == 1:
        #Choose two random routes:
        loc_R1 = (random.choice(range(0,len(testingsol))))
        loc_R2 = (random.choice(range(0,len(testingsol))))
        route1 = testingsol[loc_R1]
        route2 = testingsol[loc_R2]

        #If the routes are the same or if at least one of them is too short we return the same one back
        if route1 == route2 or (len(route1) < 3) or (len(route2) < 3):
            return original_sol
            

        #Otherwise we modify our current solution
        else:
            #Chose two random customers in each route:
            loc_C1 = (random.choice(range(1,len(route1)-1)))
            loc_C2 = (random.choice(range(1,len(route2)-1)))
            custR1 = route1[loc_C1]
            custR2 = route2[loc_C2]

            #Swap those customers:
            #temp = custR1
            route1[loc_C1] = custR2
            route2[loc_C2] = custR1

            #Create the new solution with different routes
            testingsol[loc_R1] = route1
            testingsol[loc_R2] = route2
        
        if is_feasible(instance, testingsol):
            newsolution = testingsol
        else: 
            newsolution = original_sol
        
    
    #If 2 then move one customer from a route to a new location on a different route
    elif choice == 2:
        #Choose two random routes:
        loc_R1 = (random.choice(range(0,len(testingsol))))
        loc_R2 = (random.choice(range(0,len(testingsol))))
        route1 = testingsol[loc_R1]
        route2 = testingsol[loc_R2]
        
        #If the routes are the same or if at least one of them is too short we return the same one back
        if route1 == route2 or (len(route1) < 3) or (len(route2) < 3):
            return original_sol
        
        #Otherwise we modify the route
        else:
            #Choose a random customer in the first route
            loc_Cust = (random.choice(range(1,len(route1)-1))) #The location of the customer in the list
            custR1 = route1[loc_Cust] #The number of the customer 

            #Add that customer to a random location in the second route (not first and last because that is the depot)
            place = (random.choice(range(1,len(route2)-1)))
            route2 = route2[0:place] + [custR1] + route2[place:]

            #Remove it from the first route
            route1 = route1[0:loc_Cust] + route1[loc_Cust+1:]

            #Create the new solution:
            testingsol[loc_R1] = route1
            testingsol[loc_R2] = route2
            
        if is_feasible(instance, testingsol):
            #print("It is feasible")
            newsolution = testingsol
        else: 
            newsolution = original_sol
        
    #If 3 then we switch two customers in a random route 
    elif choice == 3:
        #Choose a random route:
        loc_R1 = (random.choice(range(0,len(testingsol))))
        route1 = testingsol[loc_R1]
        
        if (len(route1) < 3):
            return original_sol

        #Choose a random customer in the first route
        loc_Cust = (random.choice(range(1,len(route1)-1))) #The location of the customer in the list
        custR1 = route1[loc_Cust] #The number of the customer 
        
        if loc_Cust > 2 and loc_Cust < len(route1)-1:
            temp = route1[loc_Cust]
            route1[loc_Cust] = route1[loc_Cust - 1]
            route1[loc_Cust - 1] = temp
        elif loc_Cust == 1 and loc_Cust < len(route1)-2: 
            temp = route1[loc_Cust]
            route1[loc_Cust] = route1[loc_Cust + 1]
            route1[loc_Cust + 1] = temp
            
        #If neither of the if or elif hold - we don't do anything and return the old route. 
        
        #Create the newsolution:
        testingsol[loc_R1] = route1
        
        if is_feasible(instance, testingsol):
            #print("It is feasible")
            newsolution = testingsol
        else: 
            newsolution = original_sol
        
    #If 4 then we take an end customer from one route and add it to the end of another route.
    elif choice == 4:
        
        #Choose two random routes:
        loc_R1 = (random.choice(range(0,len(testingsol))))
        loc_R2 = (random.choice(range(0,len(testingsol))))
        route1 = testingsol[loc_R1]
        route2 = testingsol[loc_R2]
        
        if route1 == route2 or (len(route1) < 3) or (len(route2) < 3):
            return original_sol
            
        
        #Put the last customer from route 1 at the end of route 2
        if len(route1) > 3:
            custR1 = route1[len(route1)-2]

            #Make the routes
            route1 = route1[0:len(route1)-2] + [0]
            route2 = route2[0:len(route1)-1] + [custR1] + [0]

            #Create the new solution with different routes
            testingsol[loc_R1] = route1
            testingsol[loc_R2] = route2
            newsolution = testingsol
            
        else:
            return original_sol
    
        if is_feasible(instance, testingsol):
            #print("It is feasible")
            newsolution = testingsol
        else: 
            newsolution = original_sol
        
    return newsolution 

### Step 4: Implement the Tabu Search Algorithm

In [137]:
def TabuSearch(instance):
    
    #We will initialize our first solution as the Clark and Wright solution
    currentSolution = CWHeuristic(instance)

    #Our initial best solution is the Clark and Wright solution
    bestSolution = copy.deepcopy(currentSolution)
    bestDist = total_distance(instance, bestSolution)

    #Create a Tabu List
    tabuList = [] 
    
    startTime = time.time()
    
    i = 0
    
    while True:
        
        currentCopy = copy.deepcopy(currentSolution)
        
        #We want to generate a now solution by using the neighbourhood function
        genSolution = create_neighbourhood(currentSolution)
        
        i = i + 1

        #If we have seen the solution before we continue to the next one
        if genSolution in tabuList:
            continue
        
        #We need to add the solution to the tabu list 
        tabuList.append(genSolution)

        #Need to check if the new solution is feasible     
        #If the solution is feasible we need to calculate the distance for that solution
        if is_feasible(instance, genSolution):

            #If the solution is feasible 
            genDist = total_distance(instance, genSolution)

            #If our new solution is better than the best one we update the best one
            if genDist < bestDist:
                print("New distance", genDist)
                print("Current best", bestDist)
                bestDist = genDist
                bestSolution = genSolution

            currentSolution = genSolution
        
        else:
            print("Not feasible")
            currentSolution = currentCopy
            
        #Make sure we stop running after 60 seconds
        elapsedTime = (time.time() - startTime)
        if (elapsedTime > 60):
            break

    return bestSolution, bestDist, i

### Running the Metaheuristic

In [138]:
#We will be storing our results in the following lists
routesInstances = []
bestDiste = []
totIter = []

for instance in instances: 
    newroutes, bestDist, i = TabuSearch(instance)
    routesInstances.append(newroutes)
    bestDiste.append(bestDist)
    totIter.append(i)

The code took: 0.0373072624206543 seconds to run.
Heuristic: True
The code took: 0.031783103942871094 seconds to run.
Heuristic: True
The code took: 0.04419112205505371 seconds to run.
Heuristic: True
The code took: 0.050189971923828125 seconds to run.
Heuristic: True
The code took: 0.02573680877685547 seconds to run.
Heuristic: True
The code took: 0.029829978942871094 seconds to run.
Heuristic: True
New distance 183411
Current best 183545
New distance 183335
Current best 183411
The code took: 0.021999120712280273 seconds to run.
Heuristic: True
The code took: 0.018388986587524414 seconds to run.
Heuristic: True


In [139]:
#The lowest distances found for all instances. 
print(bestDiste)

[247308, 184489, 262820, 265253, 251132, 183335, 276340, 288560]


In [140]:
#The total number of iterations for all instances. 
print(totIter)

[44246, 42544, 42679, 42772, 48131, 46002, 53651, 53554]


## Helper Functions

In [94]:
def is_feasible_trivial(instance, solution):
    """
    Checks if the solution is feasible for the instance.
    """
    # Check if all customers are served
    # Check if all customers are served
    custs = {cust for route in solution for cust in route}
    all_custs = set(range(1, instance['dimension']))

    if (custs & all_custs) != all_custs:
        print("We get to here 1")
        return False

    # Check that none of the routes exceed vehicle capacity
    capacity = instance['capacity']
    demands = instance['demand']

    for route in solution:
        if demands[route].sum() > capacity:
            print("We get here for route", route)
            return False

    # Check if all customers are served on time
    release_times = instance['release_time']
    deadlines = instance['deadline']
    service_times = instance['service_time']
    distances = instance['edge_weight']

    for route in solution:
        current_time = release_times[route].max()
        locs = [0] + route + [0]

        for idx in range(len(locs) - 1):
            pred, succ = locs[idx], locs[idx + 1]
            dist = distances[pred, succ]
            arrival = current_time + dist

            if arrival > deadlines[succ]: 
                print("We get here 3")
                return False

            current_time = arrival + service_times[succ]

    return True
            

In [135]:
def is_feasible(instance, solution):
    """
    Checks if the solution is feasible for the instance.
    """
    # Check if all customers are served
    # Check if all customers are served
    custs = {cust for route in solution for cust in route}
    all_custs = set(range(1, instance['dimension']))

    if (custs & all_custs) != all_custs:
        #print("We get to here 1")
        return False

    # Check that none of the routes exceed vehicle capacity
    capacity = instance['capacity']
    demands = instance['demand']

    for route in solution:
        if demands[route].sum() > capacity:
            #print("We get here for route", route)
            return False

    # Check if all customers are served on time
    release_times = instance['release_time']
    deadlines = instance['deadline']
    service_times = instance['service_time']
    distances = instance['edge_weight']

    for route in solution:
        current_time = release_times[route].max()
        #locs = [0] + route + [0]
        locs = route #My routes include the 0's so we need to change this line. 

        for idx in range(len(locs) - 1):
            pred, succ = locs[idx], locs[idx + 1]
            dist = distances[pred, succ]
            arrival = current_time + dist

            if arrival > deadlines[succ]: 
                #print("We get here 3")
                return False

            current_time = arrival + service_times[succ]

    return True
            

In [96]:
def total_distance(instance, solution):
    """
    Computes the total distance of the passed-in solution.
    """
    distances = instance['edge_weight']
    total = 0

    for route in solution:
        pred = [0] + route
        succ = route + [0]
        total += distances[pred, succ].sum()

    return total

In [97]:
def write_solution(fname, solution):
    """
    Writes a solution to file.
    """
    with open(fname, 'w') as fh:
        for route in solution:
            fh.write(' '.join(str(cust) for cust in route))
            fh.write('\n')

## Testing the trivial solution (TA code)

In [145]:
# The trivial solution where each customer is served using a single vehicle
# is always feasible, since we have an unlimited number of vehicles.
# The trivial solution where each customer is served using a single vehicle
# is always feasible, since we have an unlimited number of vehicles.
i = 0
for instance in instances:
    i = i + 1
    trivial_solution = [[cust] for cust in range(1, instance['dimension'])]
    print("For instance", i, "the is_feasible function returns: ",is_feasible_trivial(instance, trivial_solution))

For instance 1 the is_feasible function returns:  True
For instance 2 the is_feasible function returns:  True
For instance 3 the is_feasible function returns:  True
For instance 4 the is_feasible function returns:  True
For instance 5 the is_feasible function returns:  True
For instance 6 the is_feasible function returns:  True
For instance 7 the is_feasible function returns:  True
For instance 8 the is_feasible function returns:  True


In [144]:
i = 0
for instance in instances:
    i = i + 1
    trivial_solution = [[cust] for cust in range(1, instance['dimension'])]
    print("The total distance for instance", i, "is:", total_distance(instance, trivial_solution))

The total distance for instance 1 is: 586047
The total distance for instance 2 is: 491246
The total distance for instance 3 is: 649701
The total distance for instance 4 is: 672237
The total distance for instance 5 is: 711632
The total distance for instance 6 is: 553242
The total distance for instance 7 is: 664629
The total distance for instance 8 is: 653156


In [100]:
solution_loc = instance_loc.stem + '.sol'
write_solution(solution_loc, trivial_solution)

## Testing the Heuristic solution 

In [101]:
i = 0
for instance in instances: 
    i = i + 1
    newroutes = CWHeuristic(instance)
    print("For instance", i, "the is_feasible function returns: ", is_feasible(instance, newroutes))

For instance 1 the is_feasible function returns:  True
For instance 2 the is_feasible function returns:  True
For instance 3 the is_feasible function returns:  True
For instance 4 the is_feasible function returns:  True
For instance 5 the is_feasible function returns:  True
For instance 6 the is_feasible function returns:  True
For instance 7 the is_feasible function returns:  True
For instance 8 the is_feasible function returns:  True


In [102]:
i = 0
for instance in instances: 
    i = i + 1
    newroutes = CWHeuristic(instance)
    print("For instance", i, "the total distance is: ",total_distance(instance, newroutes))

For instance 1 the total distance is:  247308
For instance 2 the total distance is:  184489
For instance 3 the total distance is:  262820
For instance 4 the total distance is:  265253
For instance 5 the total distance is:  251132
For instance 6 the total distance is:  183545
For instance 7 the total distance is:  276340
For instance 8 the total distance is:  288560


In [103]:
#solution_loc = instance_loc.stem + '.sol'
#write_solution(solution_loc, newroutes)

## Testing the Metaheuristic solution 

In [142]:
i = 0
for instance in instances: 
    print("For instance", i, "the is_feasible function returns: ", is_feasible(instance, routesInstances[i]))
    i = i + 1

For instance 0 the is_feasible function returns:  True
For instance 1 the is_feasible function returns:  True
For instance 2 the is_feasible function returns:  True
For instance 3 the is_feasible function returns:  True
For instance 4 the is_feasible function returns:  True
For instance 5 the is_feasible function returns:  True
For instance 6 the is_feasible function returns:  True
For instance 7 the is_feasible function returns:  True


In [143]:
i = 0
for instance in instances: 
    #print("For instance", i, "the total distance for the best solution is: ", total_distance(instance, routesInstances[i]))
    print("For instance", i, "the total distance for the best solution is: ", bestDiste[i])
    i = i + 1
    
    

For instance 0 the total distance for the best solution is:  247308
For instance 1 the total distance for the best solution is:  184489
For instance 2 the total distance for the best solution is:  262820
For instance 3 the total distance for the best solution is:  265253
For instance 4 the total distance for the best solution is:  251132
For instance 5 the total distance for the best solution is:  183335
For instance 6 the total distance for the best solution is:  276340
For instance 7 the total distance for the best solution is:  288560
